# Desenvolvimento de autômatos para validação de placas de veículos do Mercosul


William Fernandes, Vitor Marques e Henrique Werneck


---


Neste projeto, foi desenvolvido um autômato finito que reconhece placas válidas de veículos registrados em países do Mercosul.

A partir de 2020, placas de veículos registrados no Mercosul devem conter um conjunto alfanumérico com quatro letras e três números, seguindo o formato AAA#A##, (onde "A" representa uma letra de A a Z e "#" representa um dígito de 0 a 9). Antes da padronização das placas entre os países do bloco, o formato era AAA####. Foi implementado um autômato finito não-determinístico que reconhece tanto credenciais no formato novo quanto no formato antigo.


In [1]:
# Instalar dependências
%pip install automathon hypothesis

  Using cached automathon-0.0.13-py3-none-any.whl
  Using cached hypothesis-6.100.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached graphviz-0.16-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
Using cached graphviz-0.16-py2.py3-none-any.whl (19 kB)
Using cached hypothesis-6.100.0-py3-none-any.whl (458 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from automathon import NFA
import string


def create_automata():
    alphabet = set(char for char in (string.ascii_uppercase + string.digits))

    letters = set(char for char in string.ascii_uppercase)
    digits = set(char for char in string.digits)
    invalid_initial_letters = {"T", "U", "V", "W", "X", "Y", "Z"}

    initial_state = "q0"
    final_states = {
        "qof",
        "qnf",
    }  # qnf -> aceite para o formato novo, qof -> aceite para o formato antigo
    delta = {
        "q0": {"": {"qo0", "qn0"}},
        "qn0": {
            **{letter: {"qn1"} for letter in (letters - invalid_initial_letters)},
        },
        "qn1": {**{letter: {"qn2"} for letter in letters}},
        "qn2": {**{letter: {"qn3"} for letter in letters}},
        "qn3": {**{digit: {"qn4"} for digit in digits}},
        "qn4": {**{letter: {"qn5"} for letter in letters}},
        "qn5": {**{digit: {"qn6"} for digit in digits}},
        "qn6": {**{digit: {"qnf"} for digit in digits}},
        "qo0": {**{letter: {"qo1"} for letter in (letters - invalid_initial_letters)}},
        "qo1": {**{letter: {"qo2"} for letter in letters}},
        "qo2": {**{letter: {"qo3"} for letter in letters}},
        "qo3": {**{digit: {"qo4"} for digit in digits}},
        "qo4": {**{digit: {"qo5"} for digit in digits}},
        "qo5": {**{digit: {"qo6"} for digit in digits}},
        "qo6": {**{digit: {"qof"} for digit in digits}},
    }

    return NFA(
        {
            "q0",
            "qo0",
            "qo1",
            "qo2",
            "qo3",
            "qo4",
            "qo5",
            "qo6",
            "qn0",
            "qn1",
            "qn2",
            "qn3",
            "qn4",
            "qn5",
            "qn6",
            "qof",
            "qnf",
        },
        alphabet,
        delta,
        initial_state,
        final_states,
    ).minimize()

In [2]:
# Testar se o autômato é valido
automata = create_automata()
assert automata.is_valid()

# Visualizar o autômato
automata.view("automata")

![automata](automata.gv.png)

In [4]:
from hypothesis.strategies import from_regex, one_of, text

# Definir estratégias de geração de placas
old_format_license_plates = from_regex(
    r"^[A-S][A-Z]{2}\d{4}$",
    fullmatch=True,
    alphabet=string.ascii_uppercase + string.digits,
)
new_format_license_plates = from_regex(
    r"^[A-S][A-Z]{2}\d[A-Z]\d{2}$",
    fullmatch=True,
    alphabet=string.ascii_uppercase + string.digits,
)
invalid_license_plates = one_of(
    from_regex(r"^[T-Z][A-Z]{2}\d{4}$"),
    text(min_size=8 ,alphabet=[char for char in (string.ascii_uppercase + string.digits)]),
    text(max_size=6, min_size=1 ,alphabet=[char for char in (string.ascii_uppercase + string.digits)]),
)

In [5]:
from hypothesis import given

In [6]:
@given(old_format_license_plates)
def test_accepts_old_format(word):
    automata = create_automata()
    assert automata.accept(word)


test_accepts_old_format()

In [7]:
@given(new_format_license_plates)
def test_accepts_new_format(word):
    automata = create_automata()
    assert automata.accept(word)


test_accepts_new_format()

In [8]:
@given(invalid_license_plates)
def test_do_not_accepts_invalid_plates(word):
    automata = create_automata()
    assert not automata.accept(word)


test_do_not_accepts_invalid_plates()